In [1]:
import numpy as np
from scipy.integrate import quad

# Bivariate normal PDF
# Equivalent to R's biv_pdf function
def biv_pdf(x, y, mu, sigma, rho, log=False):
    rho_part = 1 - rho**2
    if rho_part <= 0:
        rho_part = 0.000001  # To prevent division by zero or negative values

    factor_part = 2 * np.pi * sigma[0] * sigma[1] * np.sqrt(rho_part)
    exp_part = ((x - mu[0]) / sigma[0])**2 + ((y - mu[1]) / sigma[1])**2
    exp_part -= 2 * rho * (x - mu[0]) * (y - mu[1]) / (sigma[0] * sigma[1])
    exp_part = -exp_part / (2 * rho_part)

    if log:
        return exp_part - np.log(factor_part)
    else:
        return np.exp(exp_part) / factor_part

# Equivalent to R's dmg_int_inner function
def dmg_int_inner(x, y, mu, sigma, rho, eta, alpha, l):
    sd_x = sigma[0]
    sd_y = sigma[1]
    # Adjusted value of y
    adjusted_y = y + (mu[1] / mu[0]) * alpha * (l / eta - x)
    return biv_pdf(x, adjusted_y, mu, sigma, rho)

# Equivalent to R's dmg_int function
def dmg_int(y, mu, sigma, rho, eta, alpha, l):
    # Use scipy's quad to perform the integration
    result, _ = quad(dmg_int_inner, l, l / eta, args=(y, mu, sigma, rho, eta, alpha, l), epsabs=1e-5)
    return result

# Example usage (requires biv_pdf to be implemented)
mu = [45, 5.5]
sigma = [13, 1]
rho = .7
eta = .7
alpha = .2
l_value = 41.70649
# rho = correlation_coefficient
result = dmg_int(6, mu, sigma, rho, eta, alpha, l_value)
print(result)

0.19155548003340506


In [2]:
import numpy as np
from scipy.stats import multivariate_normal, norm

# Equivalent to R's logit function
def logit(x):
    return np.log(x / (1 - x))

# Equivalent to R's expit function
def expit(x):
    return 1 / (1 + np.exp(-x))

In [6]:
def dmg_model(samples, eta, alpha, l, mu):
    # Check if l > samples[:, 0] * eta for each row
    condition = l > samples[:, 0] * eta
    adjusted_values = samples[:, 1] - (mu[1] / mu[0]) * alpha * (l / eta - samples[:, 0])
    
    # If condition is True, calculate damage; otherwise, use undamaged value
    return np.where(condition, adjusted_values, samples[:, 1])
def int_function(y_star, mu, sigma, rho, l):
    a_l = (l - mu[0] - rho * (sigma[0] / sigma[1]) * (y_star - mu[1])) / (sigma[0] * np.sqrt(1 - rho**2))
    return norm.pdf(y_star, loc=mu[1], scale=sigma[1]) * (1 - norm.cdf(a_l))


def PFY_lik(mu, sigma, rho, eta, alpha, l, data):
    part_1 = np.sum(norm.logpdf(data[data[:, 2] == 1, 0], loc=mu[0], scale=sigma[0]))
    
    dmg_int_values = np.array([dmg_int(y, mu, sigma, rho, eta, alpha, l) for y in data[data[:, 2] == 0, 1]])
    int_function_values = np.array([int_function(y, mu, sigma, rho, 1/eta * l) for y in data[data[:, 2] == 0, 1]])
    
    part_2 = np.sum(np.log(dmg_int_values + int_function_values))
    
    return part_1 + part_2


from numpy.random import multivariate_normal

def pl_gen(mu, sigma, rho, eta, alpha, l, N):
    # Covariance matrix
    cov_matrix = np.array([
        [sigma[0]**2, sigma[0] * sigma[1] * rho],
        [sigma[0] * sigma[1] * rho, sigma[1]**2]
    ])
    # Generate samples from bivariate normal distribution
    samples = multivariate_normal(mean=mu, cov=cov_matrix, size=N)
    res = np.zeros((samples.shape[0], samples.shape[1] + 1))
    
    # Fail in the proof loading
    fail_ids = samples[:, 0] < l
    res[fail_ids, 0] = samples[fail_ids, 0]
    res[fail_ids, 2] = 1
    
    # Survive in the proof loading
    survive_ids = samples[:, 0] >= l
    res[survive_ids, 1] = dmg_model(samples[survive_ids], eta, alpha, l, mu)
    res[survive_ids, 2] = 0
    
    return res

In [30]:
import pandas as pd
bvn1 = pd.read_csv("bvn1.csv")
bvn1 = np.array(bvn1)[:,1:3]
#bvn1
def pl_gen(bvn1, eta, alpha, l, N):
    # Covariance matrix
    samples = bvn1
    res = np.zeros((samples.shape[0], samples.shape[1] + 1))
    
    # Fail in the proof loading
    fail_ids = samples[:, 0] < l
    res[fail_ids, 0] = samples[fail_ids, 0]
    res[fail_ids, 2] = 1
    
    # Survive in the proof loading
    survive_ids = samples[:, 0] >= l
    res[survive_ids, 1] = dmg_model(samples[survive_ids], eta, alpha, l, mu)
    res[survive_ids, 2] = 0
    
    return res


pf_data = pl_gen(bvn1, eta, alpha, l, N)

In [53]:
def PFY_lik(mu, sigma, rho, eta, alpha, l, data):
    # the next line is the same 
    part_1 = np.sum(norm.logpdf(data[data[:, 2] == 1, 0], loc=mu[0], scale=sigma[0]))
    # test this line 
    dmg_int_values = np.array([dmg_int(y, mu, sigma, rho, eta, alpha, l) for y in data[data[:, 2] == 0, 1]])
    # the next line is the same
    int_function_values = np.array([int_function(y, mu, sigma, rho, 1/eta * l) for y in data[data[:, 2] == 0, 1]])
    
    part_2 = np.sum(np.log(dmg_int_values + int_function_values))
    
    return part_2 + part_1
%timeit PFY_lik(mu, sigma, rho, eta, alpha, l, pf_data)


3.79 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [47]:
dmg_data = pf_data[pf_data[:, 2] == 0, 1]
dmg_int_values = np.array([dmg_int(y, mu, sigma, rho, eta, alpha, l) for y in dmg_data])
np.sum(np.log(dmg_int_values))

-101.09331377297238

In [48]:
int_function_values = np.array([int_function(y, mu, sigma, rho, 1/eta * l) for y in dmg_data])
np.sum(np.log(int_function_values))

-192.42103972561273